In [1]:
cd drive/My\ Drive/

/content/drive/My Drive


In [0]:
import numpy as np 
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [75]:
#load the csv file
df=pd.read_csv('./datasets_19_420_Iris.csv')
#Replace Species type with numbers
df.Species.replace(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], [1, 2, 3], inplace=True)
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,1
1,2,4.9,3.0,1.4,0.2,1
2,3,4.7,3.2,1.3,0.2,1
3,4,4.6,3.1,1.5,0.2,1
4,5,5.0,3.6,1.4,0.2,1


In [76]:
#Initialize Gaussian Naive Bayes Classifier
clf = GaussianNB()

# Split into labels
array = df.values
print(array[0])
X = array[:,1:5]
Y = array[:,5]

[1.  5.1 3.5 1.4 0.2 1. ]


In [77]:
# One-third of data as a part of test set
validation_size = 0.33

seed = 71
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

# Evaluation metric
scoring = 'accuracy'

# Fitting the training set
clf.fit(X_train, Y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [0]:
#Predicting for the Test Set
pred_clf = clf.predict(X_validation)

#Prediction Probability
prob_pos_clf = clf.predict_proba(X_validation)[:, 1]

In [80]:
#Create the prediction file by concatenation of the original data and predictions
#Reshaping needed to perform the concatenation
pred_clf_df = pd.DataFrame(pred_clf.reshape(50,1))
#Column renaming to indicate the predictions
pred_clf_df.rename(columns={0:'Prediction'}, inplace=True)
#reshaping the test dataset
X_validation_df = pd.DataFrame(X_validation.reshape(50,4))
pred_clf_df.head()

,Prediction
0,3.0
1,1.0
2,2.0
3,3.0
4,2.0


In [81]:
#concatenating the two pandas dataframes over the columns to create a prediction dataset
pred_outcome = pd.concat([X_validation_df, pred_clf_df], axis=1)

pred_outcome.rename(columns = {0:'SepalLengthCm', 1:'SepalWidthCm', 2:'PetalLengthCm', 3:'PetalWidthCm'}, inplace=True)
pred_outcome.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Prediction
0,6.4,2.8,5.6,2.2,3.0
1,5.1,3.8,1.6,0.2,1.0
2,5.5,2.3,4.0,1.3,2.0
3,6.2,2.8,4.8,1.8,3.0
4,5.2,2.7,3.9,1.4,2.0


In [87]:
#merging the prediction with original dataset
pred_comp = pd.merge(df,pred_outcome, on=['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm'])

#print top 10 lines of the final predictions
print((pred_comp).head(30))
print ("\n")

#Save the file to csv
pred_comp.to_csv('Predictions.csv', sep=',')

#Save the file to Excel
from pandas import ExcelWriter

writer = ExcelWriter('IrisPredictions.xlsx')
pred_comp.to_excel(writer,'Sheet1')
writer.save()

    SepalLengthCm  SepalWidthCm  ...  Species  Prediction
0             5.0           3.6  ...        1         1.0
1             5.4           3.9  ...        1         1.0
2             4.6           3.4  ...        1         1.0
3             5.0           3.4  ...        1         1.0
4             4.9           3.1  ...        1         1.0
5             4.9           3.1  ...        1         1.0
6             4.9           3.1  ...        1         1.0
7             4.8           3.0  ...        1         1.0
8             5.1           3.8  ...        1         1.0
9             5.4           3.4  ...        1         1.0
10            5.2           3.4  ...        1         1.0
11            5.2           4.1  ...        1         1.0
12            5.0           3.2  ...        1         1.0
13            5.5           3.5  ...        1         1.0
14            4.4           3.0  ...        1         1.0
15            5.1           3.8  ...        1         1.0
16            

In [88]:
#Model Performance
#setting performance parameters
kfold = model_selection.KFold(n_splits=10, random_state=seed)

#calling the cross validation function
cv_results = model_selection.cross_val_score(GaussianNB(), X_train, Y_train, cv=kfold, scoring=scoring)

#displaying the mean and standard deviation of the prediction
msg = "%s: %f (%f)" % ('NB accuracy', cv_results.mean(), cv_results.std())
print(msg)

NB accuracy: 0.960000 (0.066332)


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
